In [ ]:
!pip install yfinance
!pip install ta
!pip install mpl_finance
!pip install mplfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance
import ta
from scipy.signal import argrelextrema
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import mplfinance as mpf
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]
plt.rc('font', size=14)

In [ ]:
name = 'AXISBANK.NS'
ticker = yfinance.Ticker(name)
df = ticker.history(interval="1d",start="2022-07-02", end="2023-07-11")
df2 = df.copy()
df['Date'] = pd.to_datetime(df.index)
df['Date'] = df['Date'].apply(mpl_dates.date2num)
df = df.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]

In [ ]:
df2.reset_index(inplace=True)
df2['date'] = pd.to_datetime(df2['Date'])
df2 = df2.set_index('date')
df2 = df2.loc[:,['Open', 'High', 'Low', 'Close']]
df2.columns=  ['open', 'high', 'low','close']

## Fractal Method for S/R

In [ ]:
def isSupport(df,i):
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]
  return support
def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2]
  return resistance

In [ ]:
def using_extremas(points_to_consider):
  price_data_low = df['Low'].values
  price_data_high = df['High'].values
  local_minima_indices = argrelextrema(price_data_low, np.less, order = points_to_consider)[0]
  local_maxima_indices = argrelextrema(price_data_high, np.greater , order = points_to_consider)[0]

  return local_minima_indices, local_maxima_indices
local_minima_indices, local_maxima_indices = using_extremas(10)

In [ ]:
# plt.style.use('fivethirtyeight')
# plt.rcParams['figure.figsize'] = (20, 10)
# def get_ci(high, low, close, lookback):
#     tr1 = pd.DataFrame(high - low).rename(columns = {0:'tr1'})
#     tr2 = pd.DataFrame(abs(high - close.shift(1))).rename(columns = {0:'tr2'})
#     tr3 = pd.DataFrame(abs(low - close.shift(1))).rename(columns = {0:'tr3'})
#     frames = [tr1, tr2, tr3]
#     tr = pd.concat(frames, axis = 1, join = 'inner').dropna().max(axis = 1)
#     atr = tr.rolling(1).mean()
#     highh = high.rolling(lookback).max()
#     lowl = low.rolling(lookback).min()
#     ci = 100 * np.log10((atr.rolling(lookback).sum()) / (highh - lowl)) / np.log10(lookback)
#     return ci

# df2['ci_14'] = get_ci(df2['high'], df2['low'], df2['close'], 10)

# ax1 = plt.subplot2grid((11,1,), (0,0), rowspan = 5, colspan = 1)
# ax2 = plt.subplot2grid((11,1,), (6,0), rowspan = 4, colspan = 1)
# ax1.plot(df2['close'], linewidth = 2.5, color = '#2196f3')
# ax1.set_title('TSLA CLOSING PRICES')
# ax2.plot(df2['ci_14'], linewidth = 2.5, color = '#fb8c00')
# ax2.axhline(38.2, linestyle = '--', linewidth = 1.5, color = 'grey')
# ax2.axhline(61.8, linestyle = '--', linewidth = 1.5, color = 'grey')
# ax2.set_title('TSLA CHOPPINESS INDEX 14')
# plt.show()

In [ ]:
opening_gap_up_index = []
opening_gap_down_index = []
def using_gap():
  for i in range(1, len(df)):
    if (df['Low'][i] - df['High'][i-1]) > 0:
      if ((df['Low'][i] - df['High'][i-1]) / df['High'][i-1] ) * 100 > 0.5:
        opening_gap_up_index.append(i)
    elif (df['High'][i] - df['Low'][i-1]) <0 :
      if ((df['Low'][i-1] - df['High'][i]) / df['Low'][i-1] ) * 100 > 0.5:
        opening_gap_down_index.append(i)
using_gap()

In [ ]:
s =  np.mean(df['High'] - df['Low'])
def isFarFromLevel(l):
   return np.sum([abs(l-x) < s  for x in levels]) == 0

In [ ]:
levels = []
index = []
method = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    l = df['Low'][i]
    if isFarFromLevel(l):
      levels.append(l)
      index.append(i)
      method.append('fractal')
  elif isResistance(df,i):
    l = df['High'][i]
    if isFarFromLevel(l):
      levels.append(l)
      index.append(i)
      method.append('fractal')
  elif i in local_minima_indices:
    l = df['Low'][i]
    if isFarFromLevel(l):
      levels.append(l)
      index.append(i)
      method.append('extrema')

  elif i in local_maxima_indices:
    l = df['High'][i]
    if isFarFromLevel(l):
      levels.append(l)
      index.append(i)
      method.append('extrema')

  elif i in opening_gap_up_index:
    if isFarFromLevel(l):
      l = df['Low'][i]
      levels.append(l)
      index.append(i)
      method.append('gap_up')

  elif i in opening_gap_down_index:
    if isFarFromLevel(l):
      l = df['High'][i]
      levels.append(l)
      index.append(i)
      method.append('gap_down')



In [ ]:
s_r_df = pd.DataFrame()

In [ ]:
s_r_df['Index_value'] = index
s_r_df['Price'] = levels
s_r_df['method'] = method

In [ ]:
s_r_df

In [ ]:
def plot_all():
  fig, ax = plt.subplots()
  candlestick_ohlc(ax,df.values,width=0.6, \
                   colorup='green', colordown='red', alpha=0.8)
  date_format = mpl_dates.DateFormatter('%d %b %Y')
  ax.xaxis.set_major_formatter(date_format)
  fig.autofmt_xdate()
  fig.tight_layout()
  for i in range(len(s_r_df)):
    plt.hlines(s_r_df['Price'][i],xmin=df['Date'][s_r_df['Index_value'][i]],\
               xmax=max(df['Date']),colors='blue')
  fig.show()

In [ ]:
plot_all()

## Trendlines



In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]
plt.rc('font', size=14)


def check_trend_line(support: bool, pivot: int, slope: float, y: np.array):
    # compute sum of differences between line and prices,
    # return negative val if invalid

    # Find the intercept of the line going through pivot point with given slope
    intercept = -slope * pivot + y[pivot]
    line_vals = slope * np.arange(len(y)) + intercept

    diffs = line_vals - y

    # Check to see if the line is valid, return -1 if it is not valid.
    if support and diffs.max() > 1e-5:
        return -1.0
    elif not support and diffs.min() < -1e-5:
        return -1.0

    # Squared sum of diffs between data and line
    err = (diffs ** 2.0).sum()
    return err;


def optimize_slope(support: bool, pivot:int , init_slope: float, y: np.array):

    # Amount to change slope by. Multiplyed by opt_step
    slope_unit = (y.max() - y.min()) / len(y)

    # Optmization variables
    opt_step = 1.0
    min_step = 0.0001
    curr_step = opt_step # current step

    # Initiate at the slope of the line of best fit
    best_slope = init_slope
    best_err = check_trend_line(support, pivot, init_slope, y)
    assert(best_err >= 0.0) # Shouldn't ever fail with initial slope

    get_derivative = True
    derivative = None
    while curr_step > min_step:

        if get_derivative:
            # Numerical differentiation, increase slope by very small amount
            # to see if error increases/decreases.
            # Gives us the direction to change slope.
            slope_change = best_slope + slope_unit * min_step
            test_err = check_trend_line(support, pivot, slope_change, y)
            derivative = test_err - best_err;

            # If increasing by a small amount fails,
            # try decreasing by a small amount
            if test_err < 0.0:
                slope_change = best_slope - slope_unit * min_step
                test_err = check_trend_line(support, pivot, slope_change, y)
                derivative = best_err - test_err

            if test_err < 0.0: # Derivative failed, give up
                raise Exception("Derivative failed. Check your data. ")

            get_derivative = False

        if derivative > 0.0: # Increasing slope increased error
            test_slope = best_slope - slope_unit * curr_step
        else: # Increasing slope decreased error
            test_slope = best_slope + slope_unit * curr_step


        test_err = check_trend_line(support, pivot, test_slope, y)
        if test_err < 0 or test_err >= best_err:
            # slope failed/didn't reduce error
            curr_step *= 0.5 # Reduce step size
        else: # test slope reduced error
            best_err = test_err
            best_slope = test_slope
            get_derivative = True # Recompute derivative

    # Optimize done, return best slope and intercept
    return (best_slope, -best_slope * pivot + y[pivot])


def fit_trendlines_single(data: np.array):
    # find line of best fit (least squared)
    # coefs[0] = slope,  coefs[1] = intercept
    x = np.arange(len(data))
    coefs = np.polyfit(x, data, 1)

    # Get points of line.
    line_points = coefs[0] * x + coefs[1]

    # Find upper and lower pivot points
    upper_pivot = (data - line_points).argmax()
    lower_pivot = (data - line_points).argmin()

    # Optimize the slope for both trend lines
    support_coefs = optimize_slope(True, lower_pivot, coefs[0], data)
    resist_coefs = optimize_slope(False, upper_pivot, coefs[0], data)

    return (support_coefs, resist_coefs)



def fit_trendlines_high_low(high: np.array, low: np.array, close: np.array):
    x = np.arange(len(close))
    coefs = np.polyfit(x, close, 1)
    # coefs[0] = slope,  coefs[1] = intercept
    line_points = coefs[0] * x + coefs[1]
    upper_pivot = (high - line_points).argmax()
    lower_pivot = (low - line_points).argmin()

    support_coefs = optimize_slope(True, lower_pivot, coefs[0], low)
    resist_coefs = optimize_slope(False, upper_pivot, coefs[0], high)

    return (support_coefs, resist_coefs)











In [ ]:

data = df2.copy()
candles = data.iloc[-150:-5] # Last 30 candles in data
support_coefs_c, resist_coefs_c = fit_trendlines_single(candles['close'])
support_coefs, resist_coefs = fit_trendlines_high_low(candles['high'], candles['low'], candles['close'])

support_line_c = support_coefs_c[0] * np.arange(len(candles)) + support_coefs_c[1]
resist_line_c = resist_coefs_c[0] * np.arange(len(candles)) + resist_coefs_c[1]

support_line = support_coefs[0] * np.arange(len(candles)) + support_coefs[1]
resist_line = resist_coefs[0] * np.arange(len(candles)) + resist_coefs[1]

plt.style.use('dark_background')
ax = plt.gca()

def get_line_points(candles, line_points):
    idx = candles.index
    line_i = len(candles) - len(line_points)
    assert(line_i >= 0)
    points = []
    for i in range(line_i, len(candles)):
        points.append((idx[i], line_points[i - line_i]))
    return points

s_seq = get_line_points(candles, support_line)
r_seq = get_line_points(candles, resist_line)
s_seq2 = get_line_points(candles, support_line_c)
r_seq2 = get_line_points(candles, resist_line_c)
mpf.plot(candles, alines=dict(alines=[s_seq, r_seq, s_seq2, r_seq2], colors=['w', 'w', 'b', 'b']), type='candle', style='charles', ax=ax)
plt.show()

NameError: ignored

## Applying FIlter

In [ ]:
if trendline_price_diff_min < 3 and s_r_diff < 3:
  print('YES !!')
else:
  print("NOT MET THE CONDITION")